In [1]:
import os
import sys

pwd = os.path.sep.join(os.getcwd().split(sep=os.path.sep)[:-1])
sys.path.append(pwd)

import dataset.dataset as dataset
import model.model as model
import torch
import torch.nn as nn
import torch.functional as F
from torch.utils.data import DataLoader
import tqdm
import pandas as pd

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
result = pd.read_csv(dataset.test_csv_path)

- 모델 준비 및 학습된 가중치 로드

In [2]:
eval_model = model.classificationModel().to(device)
model_state_dict = torch.load(os.path.join(pwd, 'train/test_model.ckpt'))
eval_model.load_state_dict(model_state_dict)

eval_model.eval()

classificationModel(
  (relu): ReLU()
  (backbone): backboneModel(
    (relu): ReLU()
    (drop): Dropout(p=0.3, inplace=False)
    (conv1_1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1_1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv1_2): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (bn1_2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (res1): Sequential(
      (0): resModule(
        (relu): ReLU()
        (conv1): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
        (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (co

- 평가 데이터셋 준비

In [4]:
eval_dataset = dataset.ValidationSet(dataset.test_csv_path, 
                             dataset.test_data_path, 
                             dataset.data_transform)
eval_dataloader = DataLoader(eval_dataset, 
                             batch_size=10,
                             num_workers=4)

- inference

In [5]:
with torch.no_grad():
    for data in tqdm.notebook.tqdm(eval_dataloader):
        images, idx = data
        outputs = eval_model(images.to(device))
        predicted = torch.argmax(outputs.data, 1)
        result.loc[idx.tolist(), 'ans'] = predicted.tolist()

  0%|          | 0/1260 [00:00<?, ?it/s]

In [6]:
result.to_csv('submission.csv', index=False)